#### Functions for Regression Model

In [9]:
def regmodel(x,y):
    
    import pymc3 as pm
    import arviz as az
    
    import os
    import sys
    import glob
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
   # np.random.seed(1234)
    import pymc3 as pm
    import scipy.stats as stats
    with pm.Model() as model:
        # define priors
        a = pm.Normal('slope', 0, 16)
        b = pm.Normal('intercept', 0, 16)
        s = pm.Exponential('error', 1)

        # predictions
        obs = pm.Normal('observation', a*x + b, s, observed=y)

        # use MCMC to sample
        trace = pm.sample(return_inferencedata=True)
        pm.traceplot(trace);
        print(pm.summary(trace))
        map_estimate = pm.find_MAP(model=model)
        print(map_estimate)
    az.plot_posterior(trace)
    return model,trace


In [12]:
def ppc(prior,model,trace,*var_names):
    import pymc3 as pm
    import arviz as az
    
    import os
    import sys
    import glob
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
   # np.random.seed(1234)
    import pymc3 as pm
    import scipy.stats as stats
    varss=[]
    for i in var_names:
        varss.append(i)
    RANDOM_SEED = 58
    with model:
        ppc = pm.sample_posterior_predictive(
            trace, var_names=varss, random_seed=RANDOM_SEED
        )
    az.plot_ppc(az.from_pymc3(prior=prior,posterior_predictive=ppc, model=model))
    return ppc

In [11]:
def priorpc(model):
    import pymc3 as pm
    import arviz as az
    
    import os
    import sys
    import glob
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
   # np.random.seed(1234)
    import pymc3 as pm
    import scipy.stats as stats
    prior = pm.sample_prior_predictive(model=model)
    _, ax = plt.subplots()

    x = np.linspace(-2, 2, 50)

    for a, b in zip(prior["slope"], prior["intercept"]):
        y = a + b * x
        ax.plot(x, y, c="k", alpha=0.4)

    ax.set_xlabel("Predictor (stdz)")
    ax.set_ylabel("Mean Outcome (stdz)")
    ax.set_title("Prior predictive checks -- Flat priors")
    return prior